In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import xarray as xr
import cftime
import cartopy.crs as ccrs

# Fish catches

In [2]:
catches = pd.read_csv('data/fish_catch_europe.csv')

In [3]:
catches_long = pd.melt(catches, id_vars=['Year'], var_name='TypeLocation', value_name='Catch')
catches_long['type'] = catches_long['TypeLocation'].str.split('_').str[0]
catches_long['location'] = catches_long['TypeLocation'].str.split('_').str[1]
catches_long['logcatch'] = np.log(catches_long['Catch'] + 1)
catches_long['Year'] = catches_long['Year'].astype(int)

In [4]:
catches_long['location'].unique()

array(['Flemish', 'Dutch', 'English', 'Scottish', 'Irish', 'French',
       'GermanorthSee', 'NorwegianBohuslen', 'DanishSound',
       'DanishKattegat', 'DanishLimfjord', 'NorwegianWestCoast',
       'FrenchNFL', 'EnglishNFL', 'SpanishBasqueNFL', 'PortugueseNFL',
       'Icelandic', 'DutchIcelandic', 'FrenchNEAtlantic',
       'EnglishNEAtlantic', 'Shetland', 'Faroese', 'Norwegian'],
      dtype=object)

# ENSO

In [5]:
enso3 = pd.read_csv("data/cook2024-R15-ENSO-Rec-1500-2000.txt", delimiter="\t", comment="#", na_values="NA")
lat_lon = pd.read_csv("data/cook2024-ENSO-latlon.txt", delimiter="\t", comment="#", na_values="NA")

# Melt the ENSO DataFrame to long format
enso_melted = enso3.melt(id_vars=["Year"], var_name="gridpoint", value_name="enso")
enso_melted["gridpoint"] = enso_melted["gridpoint"].astype(int)

# Merge with the lat/lon DataFrame
enso_merged = enso_melted.merge(lat_lon, on="gridpoint")



# Convert to xarray
enso_xr = enso_merged.set_index(["Year", "lat", "lon"])["enso"].to_xarray()

enso_xr = enso_xr  - enso_xr.sel(Year=slice(1801, 1900)).mean(dim="Year")

nino3 = enso_xr.sel(lat=slice(-5, 5), lon=slice(-150, -90)).mean(dim=["lat", "lon"])
nino34 = enso_xr.sel(lat=slice(-5, 5), lon=slice(-170, -120)).mean(dim=["lat", "lon"])
nino4 = enso_xr.sel(lat=slice(-5, 5), lon = slice(-200, -150)).mean(dim=["lat", "lon"])
nino12 = enso_xr.sel(lat=slice(-10, 0), lon=slice(-90, -80)).mean(dim=["lat", "lon"])

enso = pd.merge(nino3.to_dataframe().reset_index().rename(columns={"enso": "nino3"}), nino34.to_dataframe().reset_index().rename(columns={"enso": "nino34"}), on="Year")
enso = pd.merge(enso, nino4.to_dataframe().reset_index().rename(columns={"enso": "nino4"}), on="Year")
enso = pd.merge(enso, nino12.to_dataframe().reset_index().rename(columns={"enso": "nino12"}), on="Year")

# NAO JSL

In [6]:
jsl = pd.read_excel("data/reconstructed EU JSL.xlsx")

nao_cal = pd.read_excel("data/nao_reconst.xlsx", sheet_name="Figure 2a", header=3)
nao_cal = nao_cal.rename(columns={"Time (years AD)" : "Year"})
nao_model = pd.read_excel("data/nao_reconst.xlsx", sheet_name="Figure 2b", header=3)
nao_model = nao_model.rename(columns={"Time (years AD)" : "Year"})

nao = pd.merge(nao_cal, nao_model, on="Year", suffixes=('_cal', '_model'))

nao = nao.rename(columns={"Ensemble Mean_cal" : "NAO_cal", "Ensemble Mean_model" : "NAO_model"})[["Year", "NAO_cal", "NAO_model"]]

C:\Users\emile\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
C:\Users\emile\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


# conflict

In [7]:
# Load the data
post1400 = pd.read_excel('data/Conflict-Catalog-18-vars.xlsx')
pre1400 = pd.read_excel('data/Brecke-Pre-1400-European-Conflicts.xlsx')


# Convert StartYear and EndYear to integer if not NaN
post1400['StartYear'] = post1400['StartYear'].apply(lambda x: int(x) if not pd.isna(x) else np.nan)
post1400['EndYear'] = post1400['EndYear'].apply(lambda x: int(x) if not pd.isna(x) else np.nan)

# Filter Region to be 3 and 4
post1400 = post1400[post1400['Region'].isin([3, 4])]
# Select relevant columns
post1400 = post1400[['TotalFatalities', 'MilFatalities', 'StartYear', 'EndYear']].copy()
pre1400 = pre1400[['Fatalities', 'StartYear', 'EndYear']].copy()

# Fill NaN values in TotalFatalities and MilFatalities with 0
post1400['TotalFatalities'] = post1400['TotalFatalities'].fillna(0)
post1400['MilFatalities'] = post1400['MilFatalities'].fillna(0)
post1400['Deaths'] = post1400['TotalFatalities']

pre1400['Deaths'] = pre1400['Fatalities'].fillna(0)

# Drop TotalFatalities and MilFatalities
post1400 = post1400.drop(columns=['TotalFatalities', 'MilFatalities'])

pre1400 = pre1400.drop(columns=['Fatalities'])

wars = pd.concat([pre1400,post1400])

# Initialize an empty dictionary for tracking deaths per year
death_index = {}

# Populate the index with death counts, ongoing wars, started wars, and total duration
for _, row in wars.iterrows():
    if pd.isna(row['StartYear']) or pd.isna(row['EndYear']):
        continue
    
    year_range = range(int(row['StartYear']), int(row['EndYear']) + 1)
    deaths_per_year = row['Deaths'] / len(year_range)
    duration = len(year_range)
    
    for year in year_range:
        if year in death_index:
            death_index[year]['Deaths'] += deaths_per_year
            death_index[year]['ongoing_wars'] += 1
            if year == row['StartYear']:
                death_index[year]['started_wars'] += 1
                death_index[year]['total_duration'] += duration
        else:
            death_index[year] = {
                'Deaths': deaths_per_year,
                'ongoing_wars': 1,
                'started_wars': 1 if year == row['StartYear'] else 0,
                'total_duration': duration if year == row['StartYear'] else 0
            }

# Convert dictionary to DataFrame
conflict = pd.DataFrame.from_dict(death_index, orient='index').reset_index()

conflict['Death_ratio'] = conflict['Deaths'] / conflict['ongoing_wars']
conflict.rename(columns={'index': 'Year'}, inplace=True)
conflict = conflict.sort_values(by='Year').reset_index(drop=True).fillna(0)

C:\Users\emile\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
C:\Users\emile\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


# Merge

In [8]:

df_total = pd.merge(catches_long, enso, on=['Year'])
df_total = pd.merge(df_total, jsl, on=['Year'])
df_total = pd.merge(df_total, nao, on=['Year'])
df_total['Year'] = df_total['Year'].astype(int)

df_total = pd.merge(df_total, conflict, on=['Year'], how='left')
df_total[['Deaths', 'ongoing_wars', 'started_wars', 'total_duration', 'Death_ratio']] = df_total[['Deaths', 'ongoing_wars', 'started_wars', 'total_duration', 'Death_ratio']].fillna(0)


In [9]:
df_total

,Year,TypeLocation,Catch,type,location,logcatch,nino3,nino34,nino4,nino12,JSL,NAO_cal,NAO_model,Deaths,ongoing_wars,started_wars,total_duration,Death_ratio
0,1520,Herring_Flemish,33306.0,Herring,Flemish,10.413523,0.518645,0.533434,0.508531,0.487671,-1.860499,0.427161,0.980838,2400.000000,8.0,2.0,8.0,300.000000
1,1521,Herring_Flemish,11649.0,Herring,Flemish,9.363061,-0.234432,-0.251730,-0.214205,-0.148553,0.596623,0.288109,0.505441,15633.333333,10.0,4.0,13.0,1563.333333
2,1522,Herring_Flemish,8423.0,Herring,Flemish,9.038840,-0.574152,-0.571826,-0.506192,-0.525410,0.632771,-0.366288,0.881716,70733.333333,13.0,5.0,11.0,5441.025641
3,1523,Herring_Flemish,15240.0,Herring,Flemish,9.631744,-0.148223,-0.135704,-0.147852,-0.204655,-0.514699,-0.062359,0.583854,15733.333333,11.0,1.0,2.0,1430.303030
4,1524,Herring_Flemish,16754.0,Herring,Flemish,9.726452,0.012817,-0.001080,-0.032875,-0.007543,2.452761,-0.159870,0.531597,84533.333333,6.0,1.0,2.0,14088.888889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6770,1786,Cod_Norwegian,102567.0,Cod,Norwegian,11.538281,-0.103405,-0.058572,-0.048595,-0.171002,0.107411,-1.164360,-1.500420,0.000000,1.0,0.0,0.0,0.000000
6771,1787,Cod_Norwegian,105602.0,Cod,Norwegian,11.567442,0.507160,0.514288,0.445805,0.406906,0.461622,-0.060579,-1.829530,31916.666667,2.0,1.0,6.0,15958.333333
6772,1788,Cod_Norwegian,108637.0,Cod,Norwegian,11.595777,0.586980,0.574606,0.535568,0.579579,1.614968,-0.508381,-0.705994,61916.666667,2.0,1.0,3.0,30958.333333
6773,1789,Cod_Norwegian,111672.0,Cod,Norwegian,11.623330,0.291503,0.294514,0.308725,0.331426,-1.100706,-0.568836,-1.005130,63916.666667,5.0,3.0,4.0,12783.333333


In [23]:
df_total.to_csv("data/fishcatch_enso.csv", index=False)